In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import os
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
# Load variables from .env
load_dotenv()

# Read values
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")


Wikipedia

In [60]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

Webpage

In [61]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
## Vector Embedding And Vector Store
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Use a free model from HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retrieverweb=vectordb.as_retriever()
retrieverweb

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000241F3152330>, search_kwargs={})

In [62]:
from langchain.tools.retriever import create_retriever_tool
retriever_toolweb=create_retriever_tool(retrieverweb,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

pdf

In [63]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
# Build Chroma vector store
vectordb = Chroma.from_documents(documents, embeddings)
retrieverpdf=vectordb.as_retriever()
retrieverpdf

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000241F3158EC0>, search_kwargs={})

In [64]:
from langchain.tools.retriever import create_retriever_tool
retriever_toolpdf=create_retriever_tool(retrieverweb,"attention-search",
                      "Search for information about transdormers. For any questions about transformers and attention, you must use this tool!")

In [65]:
retriever_toolpdf.name,retriever_toolweb.name

('attention-search', 'langsmith_search')

Arxiv

In [66]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [67]:
tools=[wiki,arxiv,retriever_toolweb,retriever_toolpdf]

Instead of API key from OpenAI directly we are calling gpt 4o through azure which give $250 worth free credits.

In [68]:

# Initialize AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",  # e.g. "gpt-4o-mini"
    api_version="2024-12-01-preview",
    temperature=0
)

In [69]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [70]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [71]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [72]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.
LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Start tracingGain

{'input': 'Tell me about Langsmith',
 'output': 'Langsmith is a platform designed for building production-grade applications that utilize large language models (LLMs). It focuses on providing tools for monitoring and evaluating these applications, enabling developers to deploy their solutions quickly and confidently.\n\nKey features of Langsmith include:\n\n- **Observability**: It allows users to trace the steps their application takes when handling requests, which aids in faster debugging.\n- **Evaluation**: Users can measure the quality of their applications over time, helping to build more reliable AI solutions.\n- **Prompt Testing**: Langsmith offers features for iterating on prompts, including automatic version control and collaboration tools.\n- **Framework Agnostic**: It can be used independently or in conjunction with LangChain’s open-source frameworks, such as LangChain and LangGraph.\n\nOverall, Langsmith aims to enhance the development and operational processes for applicati